In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("spotify_millsongdata.csv")

In [4]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [5]:
df.tail()

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [6]:
df.shape

(57650, 4)

In [7]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [8]:
df = df.sample(5000).drop('link',axis = 1).reset_index(drop = True)

In [9]:
df.head(10)

,artist,song,text
0,David Guetta,Toy Story,"Music can be better with you \r\nMusic, music..."
1,George Strait,Need I Say More,You're the only one and only \r\nMy heart's b...
2,Bob Seger,Louise,Louise Lyrics \r\nNow Louise she is an outlaw...
3,Loretta Lynn,Get What'cha Got And Go,A pretty boy Charlie's the name that you've be...
4,Snoop Dogg,Boom,"Get your money, whip ya hair \r\nExecutive br..."
5,Erasure,Gimme! Gimme! Gimme!,"Half past twelve, and I'm watching the late sh..."
6,Face To Face,Not Enough,Talk talk talk don't talk to me \r\nYou've go...
7,Indiana Bible College,My Adonai,Intro \r\nI will praise you \r\nI will list ...
8,Uriah Heep,Gypsy,was only seventeen \r\nI fell in love with a ...
9,Kirk Franklin,He's Able,Chorus \r\nHe's able(3x) \r\nI'know that he ...


In [10]:
df['text'][0]

'Music can be better with you  \r\nMusic, music, music, music  \r\nMusic, music, music, music  \r\nMusic, music  \r\nMusic can be better with you  \r\nMusic can be better with you  \r\nMusic can be better with you  \r\nMusic can be better with you  \r\nMusic can be better with you  \r\nMusic can be better with you  \r\nMusic can be better with you  \r\nMusic can be better with you  \r\nMusic can be better with you  \r\nMusic, music, music, music  \r\nMusic, music, music, music  \r\nMusic, music\r\n\r\n'

In [11]:
df.shape

(5000, 3)

text preprocessing

In [12]:
df['text'] = df['text'].str.lower().replace(r'^\w\s','').replace(r'\n',' ',regex = True)

In [13]:
df.tail()

,artist,song,text
4995,Steve Miller Band,Journey From Eden,listen to the blackbird sadly sing \r for you...
4996,Olivia Newton-John,Follow Me,it's by far the hardest thing i've ever done ...
4997,Radiohead,Separator,it's like i'm fallen out of bed \r from a lon...
4998,Alice Cooper,Life And Death Of The Party,"you walk into the room, everybody stares \r t..."
4999,Dan Fogelberg,Make Love Stay,now that we love \r now that the lonely night...


In [14]:
import nltk
from nltk.stem.porter import PorterStemmer

In [15]:
stemmer = PorterStemmer()

In [16]:
def token(txt):
    token = nltk.word_tokenize(txt)
    a = [stemmer.stem(w) for w in token]
    return " ".join(a)

In [17]:
token("you are beautiful")

'you are beauti'

In [18]:
df['text'].apply(lambda x: token(x))

0       music can be better with you music , music , m...
1       you 're the onli one and onli my heart 's beat...
2       louis lyric now louis she is an outlaw , in th...
3       a pretti boy charli 's the name that you 've b...
4       get your money , whip ya hair execut branch bl...
                              ...                        
4995    listen to the blackbird sadli sing for you , f...
4996    it 's by far the hardest thing i 've ever done...
4997    it 's like i 'm fallen out of bed from a long ...
4998    you walk into the room , everybodi stare the t...
4999    now that we love now that the lone night are o...
Name: text, Length: 5000, dtype: object

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
tfidf = TfidfVectorizer(analyzer = 'word',stop_words = 'english')

In [21]:
matrix = tfidf.fit_transform(df['text'])

In [22]:
similar = cosine_similarity(matrix)

In [23]:
similar[0]

array([1., 0., 0., ..., 0., 0., 0.])

In [25]:
df[df['song'] == "Separator"].index[0]

4997

Recommender function

In [26]:
def recommender(song_name):
    idx = df[df['song'] == song_name].index[0]
    distance = sorted(list(enumerate(similar[idx])),reverse = True,key = lambda x:x[1])
    song = []
    for s_id in distance[1:21]:
        song.append(df.iloc[s_id[0]].song)
    return song

In [27]:
recommender("In Your Eyes")

['Layla',
 'Sweet Wine',
 'Father O.S.A.',
 'Baby Driver',
 'Mirror, Mirror',
 'Magic Mirror',
 'El Ha Mayaan',
 'Feel It',
 'Everywhere',
 "Don't Look In My Eyes",
 'Look Out For My Love',
 'Feel So Real',
 'Love At First Sight',
 'I Like You',
 "Small Y'all",
 'Make Me Feel',
 'All Eyes',
 'Time After Time',
 'Until The End Of Time',
 'Just One Last Look']

In [28]:
import pickle

In [31]:
pickle.dump(similar,open("similarity.pkl","wb"))

In [32]:
pickle.dump(df,open("df.pkl","wb"))